In [4]:
pip install langchain openai unstructured pypdf tika faiss-cpu streamlit langchain-community


In [5]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, UnstructuredURLLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

In [6]:
%pip install --upgrade --quiet unstructured

In [7]:
from langchain_community.document_loaders import UnstructuredURLLoader

urls = [
   "https://www.spandidos-publications.com/10.3892/ijmm.2023.5232",
"https://link.springer.com/article/10.1007/s10554-022-02730-9",]

In [8]:
loader = UnstructuredURLLoader(urls=urls)

data = loader.load()

data

[Document(metadata={'source': 'https://www.spandidos-publications.com/10.3892/ijmm.2023.5232'}, page_content='Home\n\nSubmit Manuscript\n\nMy Account\n\nAdvanced Search\n\nRegister\n\nLogin\n\nOncology Letters\n\nInternational Journal of Oncology\n\nMolecular and Clinical Oncology\n\nExperimental and Therapeutic Medicine\n\nInternational Journal of Molecular Medicine\n\nBiomedical Reports\n\nOncology Reports\n\nMolecular Medicine Reports\n\nWorld Academy of Sciences Journal\n\nInternational Journal of Functional Nutrition\n\nInternational Journal of Epigenetics\n\nMedicine International\n\nInternational Journal of Molecular Medicine\n\nJournal Home\n\nCurrent Issue\n\nForthcoming Issue\n\nSpecial Issues\n\nOpen Special Issues\n\nAbout Special Issues\n\nSubmit Paper\n\nMost Read\n\nMost Cited (Dimensions)\n\nPast Two Years\n\nTotal\n\nMost Cited (CrossRef)\n\nPast Year 0\n\nTotal\n\nSocial Media\n\nPast Month\n\nPast Year\n\nTotal\n\nArchive\n\nInformation\n\nOnline Submission\n\nInform

In [9]:
%pip install -qU langchain-text-splitters

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=500,
    chunk_overlap=80,
    length_function=len,
    is_separator_regex=False,
)

In [13]:
from typing import List
from langchain.docstore.document import Document

# Assuming 'data' is a list of Document objects
docs: List[Document] = data




In [14]:
len(docs)

2

In [15]:
chunks=[]
for doc in docs:
  page_split=text_splitter.split_text(doc.page_content)
  for page_sub_split in page_split:
    metadata={"source":"HealtCare Centre"}
    doc_string=Document(page_content=page_sub_split, metadata=metadata)
    chunks.append(doc_string)

In [16]:
len(chunks)

433

In [17]:
texts = [chunk.page_content for chunk in chunks]  # Extract page_content from each Document


In [31]:
chunks[0].metadata

{'source': 'HealtCare Centre'}

In [36]:
from langchain.docstore.document import Document
# Assuming `texts` is your list of text content and `sources` is your list of corresponding source URLs or identifiers
# Create a dummy list of sources with the same length as texts for demonstration
sources = ["HealtCare Centre"] * len(texts)  # Replace with your actual source list
documents = [
    Document(page_content=text, metadata={"source": source})
    for text, source in zip(texts, sources)
]

In [18]:
#from typing import List
#from langchain.docstore.document import Document

# Assuming 'data' is a list of Document objects
#docs: List[Document] = data

# Extract the 'page_content' from each Document and join them into a single string
##text = "".join([doc.page_content for doc in docs])

# Now you can split the text using the text_splitter
#chunks = text_splitter.split_text(text)

In [37]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)



/root/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/eb02ceb48c1fdcc477ff1925c9732c379f0f0d1f/modeling_hf_nomic_bert.py:108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this 

In [38]:
pip install langchain-huggingface

In [39]:
from langchain_huggingface import HuggingFaceEmbeddings

In [40]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1",
    model_kwargs={"trust_remote_code": True}
)

In [41]:
pip install -qU langchain-community faiss-cpu

In [51]:
#vectorstore = FAISS.from_texts(texts, embedding=embeddings)
vectorstore = FAISS.from_texts(texts, embedding=embeddings, metadatas=[{"source": "Healthcare Centre"} for _ in texts])


In [52]:
file_path = "vector_index.pkl"
with open(file_path, "wb") as f:
  pickle.dump(vectorstore, f)

In [53]:
if os.path.exists(file_path):
  with open(file_path, "rb") as f:
    vectorstore = pickle.load(f)

In [54]:
!pip install transformers torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # You can replace it with "gpt2-medium", "gpt2-large", or "gpt2-xl" for larger models

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


In [56]:
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
from langchain import HuggingFacePipeline

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Create a Hugging Face pipeline
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=250)

# Wrap the Hugging Face pipeline for LangChain
langchain_pipeline = HuggingFacePipeline(pipeline=hf_pipeline)

# Example usage
# Assuming `vectorstore` is defined and configured as the retriever
chain = RetrievalQAWithSourcesChain.from_llm(llm=langchain_pipeline, retriever=vectorstore.as_retriever())


In [57]:
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7bf47af15bd0>), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'summaries'], input_types={}, partial_variables={}, template='Given the following extracted parts of a long document and a question, create a final answer with reference

In [67]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import HuggingFacePipeline
#from langchain.chains.combine_documents.stuff import StuffDocumentsChain # Not used in this example
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# Create the Hugging Face pipeline using the 'pipeline' function
hf_pipeline = pipeline(
    "text2text-generation", # Use 'text2text-generation' for sequence-to-sequence models like T5
    model=model,
    tokenizer=tokenizer,
    model_kwargs={"max_new_tokens": 512}
)

# Create the LangChain HuggingFacePipeline, passing the hf_pipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Load the QA chain with the Hugging Face pipeline using MapReduceDocumentsChain
# Remove the retriever argument and instead pass it as a keyword argument to the run method of the chain.
chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")

# When calling the chain's run method, pass the documents retrieved by the retriever
# Example:
# result = chain.run(question="your question", docs=vectorstore.as_retriever().get_relevant_documents("your question"))

In [69]:
result = chain.run(question="Why do I need both CRP and cholesterol measured?", input_documents=vectorstore.as_retriever().get_relevant_documents("Why do I need both CRP and cholesterol measured?"))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1515 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [70]:
print(result)

High levels of cholesterol and triglycerides in the blood cause the small particles of
